# Week 7 실습


Activation 함수: Sigmoid<br>
Cost 함수: Binary Cross Entrophy (BCE) / 이진 분류

<br>
<br>

H(X) = 1/(1+e^(-W^T*X))<br>
Cost(w) = -1/m * sigma{ylog(H(X))+(1-y)log(1-H(X))}

<br>
<br>

pytorch - 페이스북에서 만든 NN 라이브러리<br>
tensorflow - 구글

In [7]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [8]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score

# 데이터 읽기 함수
def load_dataset(file):
  data = np.loadtxt(file)
  print("DATA=",data)
  
  input_features = data[:,0:-1] # 모든 행에 대하여, 맨 처음부터 뒤에서 두 번째까지 모두 가져와라 // 맨 마지막 열 빼고 입력값이기 때문
  print("X=",input_features)
  
  labels = np.reshape(data[:,-1],(4,1)) # 모든 행에 대하여 마지막 열을 (4행, 1열)로 바꾸어 가져옴 // 2차원으로 바꾸고 벡터화.
  print("Y=",labels)
 
  input_features = torch.tensor(input_features, dtype=torch.float)
  labels = torch.tensor(labels, dtype=torch.float)

  return (input_features, labels)

# 모델 평가 결과 계산을 위해 텐서를 리스트로 변환하는 함수
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

x, y = load_dataset("/gdrive/My Drive/ml_colab/week7/train.txt")
# 본 강의에서는 항상 load_dataset으로 할 것

# 2 by 2 matrix -> 정규 분포



# layer 1 가중치 초기화
w1 = torch.randn(2, 2, requires_grad=True) # requires_grad: gradient 전파 -> 이 값으로 인해 바뀌도록 (?)
b1 = torch.randn(2, requires_grad=True)

print("\n[Init]\nw1 = {0}".format(tensor2list(w1)))
print("b1 = {0}".format(tensor2list(b1)))


# layer 2 가중치 초기화
w2 = torch.randn(2, 1, requires_grad=True)
b2 = torch.randn(1, requires_grad=True)

print("w2 = {0}".format(tensor2list(w2)))
print("b2 = {0}\n".format(tensor2list(b2)))



### 참고 - NA == 계산할 수 없음. (underflow 등의 이유)


# Activation 함수 설정
sigmoid = nn.Sigmoid()

# Cost 함수 - 이진분류 크로스엔트로피 비용 함수 설정 
loss_func = torch.nn.BCELoss()

# 옵티마이저 함수 (역전파 알고리즘을 수행할 함수)
optimizer = torch.optim.SGD([w1, b1, w2, b2], lr=0.2) # SGD - Stochastic Gradient Descent



""" 모델 학습 """
for epoch in range(1001):

    # H(X) 계산: forward 연산
    L2 = sigmoid(torch.add(torch.matmul(x, w1), b1)) # 1층
    hx = sigmoid(torch.add(torch.matmul(L2, w2), b2)) # 2층


    """ w, b 값 update 시작 """
    # 비용 계산
    cost = loss_func(hx, y)

    # 역전파 수행
    cost.backward()
    optimizer.step() # optimizer 실행
    """ w, b 값 update 끝 """
    # if cost 가 굉장히 작아져 0이랑 가가워지면 break


    # 100 에폭마다 비용 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())

print("\n[Learned]\nw1 = {0}".format(tensor2list(w1)))
print("b1 = {0}".format(tensor2list(b1)))
print("w2 = {0}".format(tensor2list(w2)))
print("b2 = {0}\n".format(tensor2list(b2)))


""" 모델 평가 """
# H(X) 계산: forward 연산
L2 = sigmoid(torch.add(torch.matmul(x, w1), b1))
hx = sigmoid(torch.add(torch.matmul(L2, w2), b2))


# hx는 1이 안 나옴. sigmoid 함수 결과 값이기 때문
logits = (hx > 0.5).float()
predicts = tensor2list(logits)
golds = tensor2list(y)


print("\nPRED=",predicts)
print("GOLD=",golds)
print("Accuracy : {0:f}".format(accuracy_score(golds, predicts)))

# 실행할 때마다 정확도 다름
# learning rate에 따라 정확도 달라짐

DATA= [[0. 0. 0.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
X= [[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
Y= [[0.]
 [1.]
 [1.]
 [0.]]

[Init]
w1 = [[0.0356321707367897, 0.4883567988872528], [1.2429121732711792, 0.30365464091300964]]
b1 = [-0.6879940629005432, 0.6543421149253845]
w2 = [[-0.21267671883106232], [-3.268977403640747]]
b2 = [1.6433985233306885]

0 0.7945843935012817
100 0.6995856165885925
200 0.48612305521965027
300 0.4998161196708679
400 0.5338734984397888
500 0.5009685754776001
600 0.4810628294944763
700 0.5293934941291809
800 0.507781982421875
900 0.47923383116722107
1000 0.5223032832145691

[Learned]
w1 = [[-8.16952133178711, 164.4059600830078], [-27.802165985107422, -78.1342544555664]]
b1 = [-36.44888687133789, -164.80804443359375]
w2 = [[-4.96284294128418], [114.92253112792969]]
b2 = [-1.3874603509902954]


PRED= [[0.0], [0.0], [1.0], [0.0]]
GOLD= [[0.0], [1.0], [1.0], [0.0]]
Accuracy : 0.750000
